In [ ]:
import sys
import os

sys.path.append('F:/ai-ml/itapia/backend/evo_worker')
sys.path.append('F:/ai-ml/itapia/backend/shared')

os.environ['POSTGRES_HOST'] = 'localhost'
os.environ['REDIS_HOST'] = 'localhost'

In [ ]:
import app.core.config as cfg 
cfg.PARALLEL_MULTICONTEXT_LIMIT = 20

In [ ]:
from app.dependencies import create_dependencies, get_backtest_context_manager
create_dependencies()

context_mng = get_backtest_context_manager()

In [ ]:
tickers = context_mng.get_all_ticker_contexts()

In [ ]:
from datetime import datetime
import nest_asyncio
from app.backtest.selector import BacktestPointSelector
nest_asyncio.apply()
for ticker in tickers:
    selector = BacktestPointSelector(ticker, context_mng.data_preparer)
    selector.add_significant_points(max_points=35,
                                    selector_start=datetime(2019, 9, 1),
                                    selector_end=datetime(2025, 2, 28))
    selector.add_monthly_points(day_of_month=10, max_points=20, 
                                selector_start=datetime(2019, 9, 1),
                                selector_end=datetime(2021, 12, 31))
    context_mng.add_context(ticker, selector)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

for _, context in context_mng.contexts.items():
    await context.load_data_into_memory(max_reports=50)

In [ ]:
len(context_mng.get_context('NVDA').historical_reports)

In [ ]:
from datetime import timezone

[datetime.fromtimestamp(x.generated_timestamp, timezone.utc) for x in context_mng.get_context('NVDA').historical_reports]

In [ ]:
from app.backtest.action import MEDIUM_SWING_IDEAL_MAPPER, MEDIUM_SWING_PESSIMISTIC_MAPPER
from app.backtest.evaluator import MultiContextEvaluator

evaluator = MultiContextEvaluator(contexts=context_mng.get_ready_contexts(),
                                  aggregation_method='trim-mean')

evaluator.add_evaluate_scenario(MEDIUM_SWING_IDEAL_MAPPER, weight=0.3)
evaluator.add_evaluate_scenario(MEDIUM_SWING_PESSIMISTIC_MAPPER, weight=0.7)

In [ ]:
from itapia_common.rules.nodes.registry import get_operators_by_type, get_terminals_by_type
operators_by_type = get_operators_by_type()
terminals_by_type = get_terminals_by_type()

In [ ]:
from itapia_common.schemas.entities.rules import SemanticType
from itapia_common.rules import names as nms 

for num in [-5.0, -4.0, -3.0, 3.0, 4.0, -0.9, -0.7, -0.5, 0.5, 0.7, 0.9]:
    num_str = nms.CONST_NUM(num)
    print(num_str)
    terminals_by_type[SemanticType.NUMERICAL].remove(num_str)
    

In [ ]:
from app.algorithms.pop import DominanceIndividual

In [ ]:
algorithm = 'nsga2'

In [ ]:
new_rule_name_prefix = cfg.NEW_RULE_NAME_TEMPLATE.format(algorithm=algorithm)

In [ ]:
from app.algorithms.operators.construct import RandomMaxDepthInitOperator
from itapia_common.schemas.entities.rules import SemanticType
init_opr = RandomMaxDepthInitOperator(purpose=SemanticType.DECISION_SIGNAL, max_depth=11,
                                      ind_cls=DominanceIndividual,
                                      terminals_by_type=terminals_by_type,
                                      operators_by_type=operators_by_type,
                                      new_rule_name_prefix=new_rule_name_prefix)

In [ ]:
from app.algorithms.operators.crossover import SubtreeCrossoverOperator, OnePointCrossoverOperator
subtree_crossover_opr = SubtreeCrossoverOperator(DominanceIndividual, new_rule_name_prefix)
onepoint_crossover_opr = OnePointCrossoverOperator(DominanceIndividual, new_rule_name_prefix)

from app.algorithms.operators.mutation import SubtreeMutationOperator, PointMutationOperator, ShrinkMutationOperator
subtree_mutation_opr = SubtreeMutationOperator(DominanceIndividual, max_subtree_depth=4,
                                               terminals_by_type=terminals_by_type,
                                               operators_by_type=operators_by_type,
                                               new_rule_name_prefix=new_rule_name_prefix)
point_mutation_opr = PointMutationOperator(DominanceIndividual, terminals_by_type=terminals_by_type,
                                           operators_by_type=operators_by_type,
                                           new_rule_name_prefix=new_rule_name_prefix)
shrink_mutation_opr = ShrinkMutationOperator(DominanceIndividual, terminals_by_type=terminals_by_type,
                                             new_rule_name_prefix=new_rule_name_prefix)

from app.algorithms.operators.selection import TournamentSelectionOperator
from app.algorithms.comparator import RankAndCrowdingComparator, FixedDominateComparator
selection_opr = TournamentSelectionOperator(DominanceIndividual, k=8, comparator=RankAndCrowdingComparator())

from app.algorithms.operators.replacement import NSGA2ReplacementOperator
replacement_opr = NSGA2ReplacementOperator(comparator=FixedDominateComparator())

from app.algorithms.objective import CSRSPObjectiveExtractor, CSPSPWeightedAggObjectiveExtractor
obj_extractor = CSRSPObjectiveExtractor()
weights = [0.3, 0.3, 0.2, 0.1, 0.1]
adaptive_obj_extractor = CSPSPWeightedAggObjectiveExtractor(weights)

In [ ]:
from app.algorithms.engine.nsga2 import NSGA2EvoEngine
from datetime import datetime
now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

config = {
    'pop_size': 100,
    'num_gen': 200,
    'pc': 0.85,
    'pm': 0.25,
    'lr': 0.3,
    'update_score_period': 20,
    'archive_each_gen': 90,
    'archived_weights': weights
}

engine = NSGA2EvoEngine(run_id=f'nsga2-first-test',
                        seeding_rules=None,
                        pop_size=config['pop_size'],
                        num_gen=config['num_gen'],
                        pc=config['pc'],
                        pm=config['pm'],
                        lr=config['lr'],
                        update_score_period=config['update_score_period'],
                        archive_each_gen=config['archive_each_gen'])

In [ ]:
(engine.set_evaluator(evaluator)
 .set_dominate_comparator(FixedDominateComparator())
 .set_adaptive_obj_extractor(adaptive_obj_extractor)
 .set_obj_extractor(obj_extractor)
 .set_init_opr(init_opr)
 .set_replacement_opr(replacement_opr)
 .set_selection_opr(selection_opr)
 .add_crossover_opr(subtree_crossover_opr)
 .add_crossover_opr(onepoint_crossover_opr)
 .add_mutation_opr(subtree_mutation_opr)
 .add_mutation_opr(point_mutation_opr)
 .add_mutation_opr(shrink_mutation_opr)
)

In [ ]:
from itapia_common.dblib.services import EvoService
from itapia_common.dblib.session import get_rdbms_session
service = EvoService(next(get_rdbms_session()))

In [ ]:
evo_run_ent = service.get_evo_run(engine.run_id)

In [ ]:
import pickle

fallback_state = pickle.loads(evo_run_ent.fallback_state)

In [ ]:
len(fallback_state['NSGA2EvoEngine']['archived']['population'])

In [ ]:
fallback_state['NSGA2EvoEngine']['archived']['max_population_size']

In [ ]:
from app.algorithms.pop import Population
from typing import List, Union

from app.state import Stateful, SingletonNameable

to_load_state: List[Union[Stateful, SingletonNameable]] = [
    engine,
    init_opr,
    subtree_crossover_opr,
    onepoint_crossover_opr,
    subtree_mutation_opr,
    point_mutation_opr,
    shrink_mutation_opr,
    selection_opr,
    replacement_opr
]

for obj in to_load_state:
    obj.set_from_fallback_state(fallback_state[obj.singleton_name])

In [ ]:
engine.archived.population_size

In [ ]:
engine.archived.max_population_size

In [ ]:
engine.pop.population[3].fitness

In [ ]:
front = engine.rerun(next_stop_gen=60)

In [ ]:
import pandas as pd
df = pd.DataFrame([ind.fitness for ind in front])

In [ ]:
df.describe()

In [ ]:
engine.fallback_state.keys()

In [ ]:
engine.mutation_adap._score_storage

In [ ]:
engine.crossover_adap._score_storage

In [ ]:
to_save_state = [
    engine,
    init_opr,
    subtree_crossover_opr,
    onepoint_crossover_opr,
    subtree_mutation_opr,
    point_mutation_opr,
    shrink_mutation_opr,
    selection_opr,
    replacement_opr
]

fallback_state = {obj.singleton_name: obj.fallback_state for obj in to_save_state}

In [ ]:
import pickle
pickle.dump(fallback_state, open('./a7.pkl', 'wb'))

In [ ]:
from itapia_common.schemas.entities.evo import EvoRunEntity, EvoRuleEntity
from itapia_common.schemas.entities.rules import RuleStatus

pop_rules = [EvoRuleEntity(
    evo_run_id=engine.run_id,
    metrics=ind.metrics,
    **ind.chromosome.to_entity().model_dump()
) for ind in engine.pop.population]

for ent in pop_rules:
    ent.rule_status = RuleStatus.EVOLVING
    
archived_rules = [EvoRuleEntity(
    evo_run_id=engine.run_id,
    metrics=ind.metrics,
    **ind.chromosome.to_entity().model_dump()
) for ind in engine.archived.population]

for ent in archived_rules:
    ent.rule_status = RuleStatus.DEPRECATED


In [ ]:
from itapia_common.schemas.entities.evo import EvoRunStatus

evo_run_ent = EvoRunEntity(
    run_id=engine.run_id,
    status=EvoRunStatus.RUNNING,
    algorithm=algorithm,
    config=config,
    fallback_state=pickle.dumps(fallback_state),
    rules=pop_rules
)

In [ ]:
service.save_evo_rules(archived_rules)

In [ ]:
service.save_evo_run(evo_run_ent)

In [ ]:
engine.archived.population_size